<a href="https://colab.research.google.com/github/niko-vaas/tutorial-notebooks/blob/main/TensorFlowforASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, let's download the prerequisites:

In [16]:
!pip install tensorflow
!pip install stow
!pip install tarfile
!pip install pandas
!pip install tqdm
!pip install urllib.request
!pip install mltu
!pip install models==0.9.1
!pip install configs

ERROR: Could not find a version that satisfies the requirement tarfile (from versions: none)
ERROR: No matching distribution found for tarfile
ERROR: Could not find a version that satisfies the requirement urllib.request (from versions: none)
ERROR: No matching distribution found for urllib.request
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


You may notice a couple of errors pertaining to `tarfile` and `urllib.request`. This is because they come with Python3.

Now, let's import everything:

In [20]:
import tensorflow as tf

import os
import stow
import tarfile # dataset is in .tar.gz format
import pandas as pd
from tqdm import tqdm
from urllib.request import urlopen # to get dataset
from io import BytesIO

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from mltu.preprocessors import WavReader

from mltu.tensorflow.dataProvider import DataProvider
from mltu.transformers import LabelIndexer, LabelPadding, SpectrogramPadding
from mltu.tensorflow.losses import CTCloss
from mltu.tensorflow.callbacks import Model2onnx, TrainLogger
from mltu.tensorflow.metrics import CERMetric, WERMetric

# from torch import train_model
# from configs import ModelConfigs

The next step is to dowload our dataset, which is hosted on the internet. Before we do that, however, we're going to define a function that will make this significantly easier.

In [7]:
def download_and_unzip(url, extract_to="Datasets", chunk_size=1024*1024):
    http_response = urlopen(url)

    data = b""
    iterations = http_response.length // chunk_size + 1
    for _ in tqdm(range(iterations)):
        data += http_response.read(chunk_size)

    tarFile = tarfile.open(fileobj=BytesIO(data), mode="r|bz2")
    tarFile.extractall(path=extract_to)
    tarFile.close()

Now, let's download (and load) our dataset, the [LJS dataset](https://keithito.com/LJ-Speech-Dataset/). This is a large (2.6 GB) dataset which primarily contains recordings of people reading stuff. Sit tight, because this step takes a *long* time.

In [21]:
dataset_path = os.path.join("Datasets", "LJSpeech-1.1")
if not os.path.exists(dataset_path):
    download_and_unzip("https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2", extract_to="Datasets")

100%|██████████| 2622/2622 [53:08<00:00,  1.22s/it]
